# Import needed packages

In [6]:
import geopandas as gpd
import matplotlib.patches as mpatches
import contextily as ctx
import matplotlib.pyplot as plt
import pandas as pd
import os
#import functions as fn
import rasterio
from rasterio.plot import show
from shapely.ops import unary_union
from shapely.validation import make_valid
from shapely.ops import nearest_points
import numpy as np

In [7]:
year = 2005 #2005 is the base year for the matching process 

In [8]:
#tcc_path = os.path.join(data_folder, f'TCC_Adjusted_Images/TCC_PlantAdjusted_30m_{year}.tif')

In [9]:
%load_ext autoreload
%autoreload 2

In [ ]:
#Clip the tcc raster to exclude areas already designated as CF
clipped_raster_path = 'clipped_tcc.tif'
root_path ='DISES/Proximity/' #https://drive.google.com/open?id=1x8AejXYD7obyORUxwCOMbnfWt_0D-2fx&usp=drive_fs
bpb_path = 'DISES/batched-predictions-branch/' #https://drive.google.com/open?id=1LPejWJ2_0OSd_lDuPcsioi8Vq5d1WQPX&usp=drive_fs



In [5]:
import rasterio
import numpy as np
from scipy.ndimage import label, measurements
import geopandas as gpd
from shapely.geometry import Polygon

def filter_raster_by_percentage_and_area(raster_path, percentage, min_area_hectares, return_shapefile=False, output_shapefile_path=None):
    """
    Identifies zones in a raster where pixels exceed a percentage threshold and
    have a minimum contiguous area in hectares. Optionally returns a shapefile of the zones.

    Parameters:
        raster_path (str): Path to the input raster file.
        percentage (float): Percentage threshold for pixel values.
        min_area_hectares (float): Minimum area of zones to identify, in hectares.
        return_shapefile (bool): If True, returns a shapefile of the zones.
        output_shapefile_path (str): Path to save the output shapefile (required if return_shapefile is True).

    Returns:
        numpy.ndarray or None: Binary array with 1 indicating valid zones and 0 otherwise (if return_shapefile is False).
    """
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)  # Read the first band
        transform = src.transform

        # Calculate pixel area in square meters from raster resolution
        pixel_width = src.res[0]  # Width of a pixel in meters
        pixel_height = src.res[1]  # Height of a pixel in meters
        pixel_area_m2 = pixel_width * pixel_height

        # Create a binary mask of pixels exceeding the percentage threshold
        binary_mask = raster_data > percentage

        structure_4 = np.array([[0, 1, 0], 
                        [1, 1, 1], 
                        [0, 1, 0]])
        labeled_array, num_features = label(binary_mask, structure=structure_4)  # Use 4-connectivity


        # Calculate pixel areas in hectares
        min_area_pixels = (min_area_hectares * 10_000) / pixel_area_m2

        # Initialize an output mask for valid zones
        valid_zones = np.zeros_like(raster_data, dtype=np.uint8)

        geometries = []

        # Vectorized region processing
        region_areas = measurements.sum(binary_mask, labeled_array, index=np.arange(1, num_features + 1))
        large_regions = np.where(region_areas >= min_area_pixels)[0] + 1

        for region_label in large_regions:
            region = labeled_array == region_label
            valid_zones[region] = 1

            if return_shapefile:
                # Create a polygon for the region
                mask = (labeled_array == region_label).astype(np.uint8)
                shapes = rasterio.features.shapes(mask, transform=transform)
                for shape, value in shapes:
                    if value == 1:
                        geometries.append(Polygon(shape['coordinates'][0]))

        if return_shapefile:
            if not output_shapefile_path:
                raise ValueError("Output shapefile path must be provided if return_shapefile is True.")

            # Create a GeoDataFrame from the geometries
            gdf = gpd.GeoDataFrame({"geometry": geometries}, crs=src.crs)
            gdf.to_file(output_shapefile_path, driver="ESRI Shapefile")

            return None

    return valid_zones

In [ ]:
# Identify forested areas
fa_shape_path = os.path.join(bpb_path,'data/KHM/forested_areas', 'fa_70pc_129has_4conn.shp')
filter_raster_by_percentage_and_area(clipped_raster_path, percentage=70, min_area_hectares=129, return_shapefile=True, output_shapefile_path=fa_shape_path)

In [ ]:
#Calculate avergae average level of TC in CFs:
avg_value = fn.calculate_average_raster_value(tcc_path, cfs_path)
print(f"Average Value: {avg_value}")

# Find treatment and control villages

- First we find the villages closest to each community forest and we consider those as treatment.
- Then we take the treatment villages out and apply different approaches to find the control villages.

In [ ]:
#cfs_path = '/Users/Daniel/Library/CloudStorage/GoogleDrive-dwiesner@sig-gis.com/.shortcut-targets-by-id/1Y83sGckPnURtqsg-y0FRgK1eOjNe7TSz/DISES shared/Boundary and forest cover change sub-groups folder/Boundary Data/Cambodia/CF Boundary Cambodia/Cambodia Shapefile 5-2-2023/All Cambodia 599 CF 5-2-2023/All Cambodia 599 CF 5-2-2023.shp'

cfs_path = 'data/All Cambodia 599 CF 5-2-2023/All Cambodia 599 CF 5-2-2023.shp'
cfs = gpd.read_file(cfs_path)

In [4]:
crs = cfs.crs

In [ ]:
#fa_shape_path = os.path.join('/Users/Daniel/Library/CloudStorage/GoogleDrive-dwiesner@sig-gis.com/My Drive/DISES/batched-predictions-branch/data/KHM/forested_areas', 'fa_70pc_129has_4conn.shp')
fa_shape_path = os.join.path(root_path, 'data/fa_70pc_129has_4conn.shp')
fas = gpd.read_file(fa_shape_path)
fas = fas.to_crs(crs)

In [ ]:
#Import villages shapefile 
#vlgs_path = os.path.join('/Users/Daniel/Library/CloudStorage/GoogleDrive-dwiesner@sig-gis.com/.shortcut-targets-by-id/1UFnD8ofHi_4YXqnGCH4FLW8bapLdp4qX/DISES SIG/Dataset Information/Shapefiles/Cambodia_Admin-2015/Villages.shp')
vlgs_path = os.join.path(root_path, 'data/KHM-Villages/Villages.shp')
vlgs = gpd.read_file(vlgs_path)
vlgs['PHUMCODE'] = vlgs['PHUMCODE'].astype(str)
#vlgs = fn.exclude_zero_coordinates(vlgs)
vlgs = vlgs.to_crs(crs) #Check crs to make sure they are the same

In [ ]:
# Import CPA shapefile

cpa = gpd.read_file(os.join.path(root_path, 'data/Community_Protected_Areas_Cambodia/CPA_Shape_31_Aug_2022.shp'))

# Add an ID column to the CPA starting from 1 to the total number of rows

cpa['CPA_id'] = range(1, len(cpa) + 1)

cpa = cpa.to_crs('EPSG:3148')

In [ ]:
# outputs path

outputs_path = os.join.path(root_path, 'data/outputs')

## 1. Find the nearest villages (Daniel's version)

In [ ]:
# Helper function to find the nearest village to a geometry
def find_nearest(village_gdf, target_geom):
    # Find the nearest point in the village dataset to the target geometry
    nearest = village_gdf.iloc[village_gdf.distance(target_geom).idxmin()]
    return nearest

# 1. Find treatment villages (nearest to each CF)
treatment_villages = []
for _, cf_geom in cfs.iterrows():
    nearest_village = find_nearest(vlgs, cf_geom.geometry)
    treatment_villages.append(nearest_village)

treatment_villages_gdf = gpd.GeoDataFrame(treatment_villages, crs=crs)

treatment_path = os.path.join(bpb_path, 'data/KHM/treatment_contor_proximity/treatment_vlgs.shp')

treatment_villages_gdf.to_file(treatment_path)

In [23]:
#Exclude treatment villages from the dataset
vlgs2 = vlgs[~vlgs['PHUMCODE'].isin(treatment_villages_gdf['PHUMCODE'])]

In [40]:
vlgs2 = vlgs2.reset_index(drop=True)

## 1. Find the nearest villages version 2.0

This version adds information of the CFs to the treatment villages

In [9]:
# Helper function to find the nearest village to a geometry
def find_nearest(village_gdf, target_geom):
    # Find the nearest point in the village dataset to the target geometry
    nearest = village_gdf.iloc[village_gdf.distance(target_geom).idxmin()]
    return nearest

In [10]:
# Add an ID column to the CFs starting from 1 to the total number of rows

cfs['cf_id'] = range(1, len(cfs) + 1)


In [ ]:
#cfs.to_file(os.join.path(root_path, data/All Cambodia 599 CF 5-2-2023/All Cambodia 599 CF 5-2-2023_v2.shp')

In [11]:
%%time
treatment_villages = []
for _, cf in cfs.iterrows():
    nearest_village = find_nearest(vlgs, cf.geometry)
    
    # Keep all nearest village info and also cf's attributes (excluding its geometry)
    merged_info = nearest_village.to_dict()  # Convert village info to dictionary
    cf_info = cf.drop("geometry").to_dict()  # Convert CF info, excluding geometry
    
    # Add CF info to village info
    merged_info.update(cf_info)
    
    treatment_villages.append(merged_info)

CPU times: user 9.54 s, sys: 48.7 ms, total: 9.58 s
Wall time: 9.59 s


In [12]:
# Create GeoDataFrame with the combined data
treatment_villages_gdf = gpd.GeoDataFrame(treatment_villages, crs=crs)
treatment_villages_gdf = treatment_villages_gdf[['NUMBER', 'PHUMCODE', 'VILLAGE', 'UniqueID', 'CF_Name_En','cf_id', 'geometry']].copy()
# Create a new column to identify the treatment villages in further merge processes 
treatment_villages_gdf['Treatment'] = 1

In [13]:
# Exclude treatment villages from the full village dataset
vlgs2 = vlgs[~vlgs['PHUMCODE'].isin(treatment_villages_gdf['PHUMCODE'])].reset_index(drop=True)

In [14]:
# Create a df with the equivalences of the CF codes and the village codes

cf_id_dict = treatment_villages_gdf[['PHUMCODE', 'cf_id']].copy()
cf_id_dict.rename({'PHUMCODE':'treatm_id'}, axis=1, inplace=True)

In [ ]:
#treatment_villages_gdf.to_file(os.join.path(root_path, data/treatment_contor_proximity/treatment_vlgs_v2.shp')

In [ ]:
#treatment_villages_gdf_v2.to_excel(os.join.path(root_path, data/outputs/treatment_villages/treatment_villages_v2.xlsx', index=False)

### Check if the closest village to each CF is the one named in the db of CFs-Villages

In [ ]:
# import db of CFs-Villages
cfs_villages = pd.read_excel(os.join.path(root_path, data/CF and CPA database v2.xlsx', sheet_name='CF data')

# Filter only needed variables
cfs_villages = cfs_villages[['CF/CPA name ','Village name']].copy()


In [19]:
# Split the column into multiple columns
df_split = cfs_villages['Village name'].str.split(',', expand=True) 

df_split.columns = [f'Village_{i+1}' for i in range(df_split.shape[1])]

# Concatenate with original DataFrame, dropping the old 'Values' column
df_final = pd.concat([cfs_villages.drop(columns=['Village name']), df_split], axis=1)

In [20]:
df_final.nunique()

CF/CPA name     688
Village_1       423
Village_2        24
Village_3        15
Village_4         8
Village_5         3
Village_6         1
Village_7         1
dtype: int64

In [21]:
print('There are 688 community forests and 475 villages')

There are 688 community forests and 475 villages


In [22]:
# Merge both treatment villages and the db of CFs-Villages

villages_merge = treatment_villages_gdf.merge(df_final, left_on='CF_Name_En', right_on='CF/CPA name ', how='left', indicator=True)

In [23]:
# Add a new column that checks if values in 'column1' are the same as 'column2'
villages_merge['same'] = villages_merge['VILLAGE'] == villages_merge['Village_1']
villages_merge['same'].value_counts()

same
False    534
True      76
Name: count, dtype: int64

In [24]:
# See only the villages contained in both datasets
villages_merge[villages_merge['_merge']=='both']['PHUMCODE'].nunique()

352

In [25]:
print('Only 352 treatment villages are also in the db of villages')

Only 352 treatment villages are also in the db of villages


In [26]:
print('Only 76 villages are the same in both datasets')

Only 76 villages are the same in both datasets


In [27]:
cols_to_nan = ['Village_1', 'Village_2', 'Village_3','Village_4', 'Village_5', 'Village_6', 'Village_7']

In [28]:
villages_merge[cols_to_nan] = (villages_merge[cols_to_nan].fillna(np.nan))

/var/folders/rg/s4kgtnsx69j8kmy5pyg5ngrm0000gn/T/ipykernel_34540/2348779144.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  villages_merge[cols_to_nan] = (villages_merge[cols_to_nan].fillna(np.nan))


In [29]:
villages_merge['_merge'] = villages_merge['_merge'].astype(str)

In [219]:
#villages_merge.to_excel(outputs_path + '/villages_comparison/villages_comparison.xlsx', index=False)

## 2. Find control villages (nearest to each CF)

### Relevant functions

In [15]:
# Function to calculate distance to nearest geometry and add it as a column
def calculate_distances(gdf1, gdf2, name):
    """
    Calculates the distance from each geometry in gdf1 to the closest geometry in gdf2.

    Parameters:
        gdf1 (GeoDataFrame): GeoDataFrame containing geometries to calculate distances for.
        gdf2 (GeoDataFrame): GeoDataFrame containing target geometries to calculate distances to.

    Returns:
        GeoDataFrame: Updated gdf1 with a new column 'nearest_distance' containing distances in meters.
    """
    # Ensure both GeoDataFrames have the same CRS
    if gdf1.crs != gdf2.crs:
        gdf2 = gdf2.to_crs(gdf1.crs)

    # Calculate distances to the nearest geometry in gdf2 for each geometry in gdf1
    distances = gdf1.geometry.apply(lambda geom: gdf2.geometry.distance(geom).min())

    # Add distances as a new column in gdf1
    gdf1[name] = distances

    return gdf1

### Define forests

In [16]:
from shapely.validation import make_valid
# Validate geometries to avoid topology errors
cfs['geometry'] = cfs['geometry'].apply(make_valid)
fas['geometry'] = fas['geometry'].apply(make_valid)

In [17]:
gdf1 = cfs[['CF_Code', 'geometry']]
gdf1.columns = ['FID', 'geometry']

In [18]:
gdf2 = fas.copy(deep=True)

In [19]:
forests = pd.concat([gdf1, gdf2])
forests = forests.reset_index(drop=True)

### Import Treatment villages

(This corresponds to treatmente villages already created with 1. Find the nearest villages)

In [ ]:
# Load data
treatment_path = os.join.path(root_path, 'data/treatment_contor_proximity/treatment_vlgs.shp')  # Treatment villag
treatment_villages_gdf = gpd.read_file(treatment_path)

# Create a new column to identify the treatment villages in further merge processes 
treatment_villages_gdf['Treatment'] = 1

### Original Approach
Find the villages closest to either a CF or an FA, and consider them control. 

In [46]:
# 1. Find treatment villages (nearest to each CF)
control_villages = []

for _, f_geom in forests.iterrows():
    nearest_village = find_nearest(vlgs2, f_geom.geometry)
    control_villages.append(nearest_village)

In [49]:
control_villages_gdf = gpd.GeoDataFrame(control_villages, crs=crs)

In [ ]:
control_path = os.path.join(bpb_path, 'data/KHM/treatment_contor_proximity/control_vlgs.shp')
control_villages_gdf.to_file(control_path)

In [75]:
#Create a gdf with treatment and control villages only
#treatment_villages_gdf['Treatment'] = 1
control_villages_gdf['Treatment'] = 0

vlgs3 = pd.concat([treatment_villages_gdf, control_villages_gdf])

vlgs3 = vlgs3.reset_index(drop=True)

vlgs3 = vlgs3[['NUMBER', 'PHUMCODE', 'VILLAGE', 'geometry',
       'Treatment']]

#### Calculate distances from treatment and control villages

In [77]:
vlgs3 = calculate_distances(vlgs3, cfs, 'dist_cf_m')

In [79]:
vlgs3 = calculate_distances(vlgs3, fas, 'dist_fa_m')

In [80]:
vlgs3 = calculate_distances(vlgs3, forests, 'dist_forest_m')

In [89]:
vlgs3.drop(['NUMBER', 'PHUMCODE'], axis=1).groupby('Treatment').describe().round(2).transpose()

Treatment                    0          1
dist_cf_m     count     982.00     598.00
              mean     9852.79    1601.89
              std     17790.72    1364.02
              min         0.00       0.00
              25%      1838.67     667.67
              50%      3015.86    1268.91
              75%      9182.92    2169.25
              max    102656.87    9613.98
dist_fa_m     count     982.00     598.00
              mean    16360.28   23963.89
              std     18446.15   20139.60
              min         0.00     231.33
              25%      4181.37   10051.96
              50%      9899.75   18191.17
              75%     21988.08   32477.26
              max    164809.80  165894.83
dist_forest_m count     982.00     598.00
              mean     2980.22    1565.88
              std      2303.01    1312.31
              min         0.00       0.00
              25%      1458.46     667.01
              50%      2443.00    1256.55
              75%      3747.90    2146.96
              max     15514.30    9613.98

### New Approaches

### Approach 2

From the CFs, pick villages that are at least 5km away but less than 10kms away as control

In [40]:
%%time
# Step 1: Create buffers for 5 km and 10 km
cfs_buffer_5km = cfs.copy()
cfs_buffer_5km['geometry'] = cfs_buffer_5km.geometry.buffer(5000)

cfs_buffer_10km = cfs.copy()
cfs_buffer_10km['geometry'] = cfs_buffer_10km.geometry.buffer(10000)

CPU times: user 382 ms, sys: 33 ms, total: 415 ms
Wall time: 439 ms


In [41]:
# Step 2: Find villages within the 10 km buffer
villages_within_10km = gpd.sjoin(vlgs2, cfs_buffer_10km, predicate='within')

In [42]:
# Step 3: Find villages within the 5 km buffer
villages_within_5km = gpd.sjoin(vlgs2, cfs_buffer_5km, predicate='within')

In [43]:
# Step 4: Exclude villages within the 5 km buffer from those within 10 km
control_villages_appch2 = villages_within_10km[~villages_within_10km['PHUMCODE'].isin(villages_within_5km['PHUMCODE'])].reset_index(drop=True)

In [44]:
# Step 5: Calculate distance from the village to the each community forest whose buffer contains the village
    # This seeks to select the CFs that is closest to each village 

    # Merge the two geometries (villages and community forests)
control_villages_appch2 = control_villages_appch2.merge(cfs[['cf_id','geometry']], on="cf_id", suffixes=("_village", "_forest"))

    # Convert to GeoDataFrame 
control_villages_appch2 = gpd.GeoDataFrame(control_villages_appch2, geometry="geometry_village")

    # Calculate distance between both geometries 
control_villages_appch2["distance_to_forest"] = control_villages_appch2["geometry_village"].distance(gpd.GeoSeries(control_villages_appch2["geometry_forest"], crs=control_villages_appch2.crs))

In [45]:
# Step 6: Keep the village with the shortest distance to a CF

control_villages_appch2 = control_villages_appch2.sort_values(by=["cf_id", "distance_to_forest"]).drop_duplicates(subset='PHUMCODE', keep='first').reset_index(drop=True)

In [46]:
# Step 7: Arrange dataset

    # Add treatment village id
control_villages_appch2 = control_villages_appch2.merge(cf_id_dict, on='cf_id', how='left')

    # Select only necessary variables 

control_villages_appch2 = control_villages_appch2[['PHUMCODE', 'VILLAGE', 'cf_id', 'distance_to_forest', 'treatm_id', 'geometry_village']].copy()

    # Rename column to standardize
control_villages_appch2.rename({'geometry_village':'geometry', 'distance_to_forest':'dist_forst'}, axis=1, inplace=True)

    # Convert to GeoDataFrame 
control_villages_appch2 = gpd.GeoDataFrame(control_villages_appch2, geometry="geometry", crs=vlgs.crs)

#### Calculate distances from treatment and control villages

In [47]:
#Create a gdf with treatment and control villages only
control_villages_appch2['Treatment'] = 0

vlgs_appch2 = pd.concat([treatment_villages_gdf, control_villages_appch2])

vlgs_appch2 = vlgs_appch2.reset_index(drop=True)

vlgs_appch2 = vlgs_appch2[['PHUMCODE', 'VILLAGE','Treatment','treatm_id', 'cf_id','geometry']]

In [48]:
%%time
# Calculate the distance between the village and the community forest
vlgs_appch2 = calculate_distances(vlgs_appch2, cfs, 'dist_cf_m')

# Calculate the distance between the village and the FAS
vlgs_appch2 = calculate_distances(vlgs_appch2, fas, 'dist_fa_m')

# Calculate the distance between the village and any type of forest
vlgs_appch2 = calculate_distances(vlgs_appch2, forests, 'dist_forest_m')

CPU times: user 1min 13s, sys: 429 ms, total: 1min 14s
Wall time: 1min 15s


In [49]:
vlgs_appch2.drop(['PHUMCODE', 'cf_id', 'treatm_id'], axis=1).groupby('Treatment').describe().round(2).transpose()

Treatment                    0          1
dist_cf_m     count    2486.00     598.00
              mean     7288.56    1601.89
              std      1429.25    1364.02
              min      4995.66       0.00
              25%      6039.79     667.67
              50%      7158.69    1268.91
              75%      8488.22    2169.25
              max      9992.29    9613.98
dist_fa_m     count    2486.00     598.00
              mean    41097.08   23963.89
              std     28463.81   20139.60
              min         0.00     231.33
              25%     23006.50   10051.96
              50%     38602.40   18191.17
              75%     52297.43   32477.26
              max    173696.29  165894.83
dist_forest_m count    2486.00     598.00
              mean     7123.59    1565.88
              std      1589.73    1312.31
              min         0.00       0.00
              25%      5923.86     667.01
              50%      7022.66    1256.55
              75%      8378.98    2146.96
              max      9992.29    9613.98

#### CPA
CPA tag (dummy) and have distances of treatment and control villages to CPAs

In [45]:
# Step 1: Sjoin of villages and CPAs
cpa_apprch2 = gpd.sjoin(vlgs_appch2, cpa[['CPA_id', 'CPAName_Eg', 'geometry']], predicate='within')

In [46]:
# Step 2: Create a dummy to villages that are within CPAs

vlgs_appch2['CPA_dummy'] = np.where(vlgs_appch2['PHUMCODE'].isin(cpa_apprch2['PHUMCODE']), 1, 0)

In [47]:
# Step 3: Calculate distances

vlgs_appch2 = calculate_distances(vlgs_appch2, cpa, 'dist_cpa_m')

In [48]:
vlgs_appch2['treatm_id'] = vlgs_appch2['treatm_id'].fillna(vlgs_appch2['PHUMCODE'])

#### Export files apprch 2

In [247]:
#cfs_buffer_10km.to_file(outputs_path + '/cfs_buffer_10km/cfs_buffer_10km.shp')

#cfs_buffer_5km.to_file(outputs_path + '/cfs_buffer_5km/cfs_buffer_5km.shp')

#control_villages_appch2.to_file(outputs_path + '/control_villages_appch2/control_villages_appch2.shp')

vlgs_appch2.to_file(outputs_path + '/control_villages_appchA/vlgs_appchA.shp')


/var/folders/rg/s4kgtnsx69j8kmy5pyg5ngrm0000gn/T/ipykernel_99535/3777793634.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vlgs_appch2.to_file(outputs_path + '/control_villages_appchA/vlgs_appchA.shp')


### Approach 3

Pool all villages within 5km of CFs as control (CFs)

In [49]:
# Step 1: Perform a spatial join to find villages within the buffer
control_villages_appch3 = gpd.sjoin(vlgs2, cfs_buffer_5km, predicate='within')

In [50]:
# Step 2: Calculate distance from the village to the each community forest whose buffer contains the village

    # Merge the two geometries (villages and community forests)
control_villages_appch3 = control_villages_appch3.merge(cfs[['cf_id','geometry']], on="cf_id", suffixes=("_village", "_forest"))

    # Convert to GeoDataFrame 
control_villages_appch3 = gpd.GeoDataFrame(control_villages_appch3, geometry="geometry_village")

    # Calculate distance between both geometries 
control_villages_appch3["distance_to_forest"] = control_villages_appch3["geometry_village"].distance(gpd.GeoSeries(control_villages_appch3["geometry_forest"], crs=control_villages_appch3.crs))

In [51]:
# Step 6: Drop duplicated villages with the shortest distance to a CF

control_villages_appch3 = control_villages_appch3.sort_values(by=["cf_id", "distance_to_forest"]).drop_duplicates(subset='PHUMCODE', keep='first').reset_index(drop=True)

In [52]:
# Add treatment village id
control_villages_appch3 = control_villages_appch3.merge(cf_id_dict, on='cf_id', how='left')

# Select only necessary variables 

control_villages_appch3 = control_villages_appch3[['PHUMCODE', 'VILLAGE', 'cf_id', 'distance_to_forest', 'treatm_id', 'geometry_village']].copy()

# Rename column to standardize
control_villages_appch3.rename({'geometry_village':'geometry', 'distance_to_forest':'dist_forst'}, axis=1, inplace=True)

In [53]:
    # Convert to GeoDataFrame 
control_villages_appch3 = gpd.GeoDataFrame(control_villages_appch3, geometry="geometry", crs=vlgs.crs)

#### Calculate distances from treatment and control villages

In [54]:
#Create a gdf with treatment and control villages only
control_villages_appch3['Treatment'] = 0

vlgs_appch3 = pd.concat([treatment_villages_gdf, control_villages_appch3])

vlgs_appch3 = vlgs_appch3.reset_index(drop=True)

vlgs_appch3 = vlgs_appch3[['PHUMCODE', 'VILLAGE','Treatment','treatm_id', 'cf_id','geometry']]

In [55]:
%%time
# Calculate the distance between the village and the community forest
vlgs_appch3 = calculate_distances(vlgs_appch3, cfs, 'dist_cf_m')

# Calculate the distance between the village and the FAS
vlgs_appch3 = calculate_distances(vlgs_appch3, fas, 'dist_fa_m')

# Calculate the distance between the village and any type of forest
vlgs_appch3 = calculate_distances(vlgs_appch3, forests, 'dist_forest_m')

CPU times: user 1min 4s, sys: 497 ms, total: 1min 4s
Wall time: 1min 6s


In [57]:
vlgs_appch3.drop(['PHUMCODE', 'cf_id', 'treatm_id'], axis=1).groupby('Treatment').describe().round(2).transpose()

Treatment                    0          1
dist_cf_m     count    2045.00     598.00
              mean     3080.04    1601.89
              std      1224.48    1364.02
              min         0.00       0.00
              25%      2191.48     667.67
              50%      3157.62    1268.91
              75%      4133.99    2169.25
              max      4998.12    9613.98
dist_fa_m     count    2045.00     598.00
              mean    35880.99   23963.89
              std     28546.22   20139.60
              min         0.00     231.33
              25%     16613.85   10051.96
              50%     30946.80   18191.17
              75%     48373.67   32477.26
              max    171048.62  165894.83
dist_forest_m count    2045.00     598.00
              mean     3068.65    1565.88
              std      1226.33    1312.31
              min         0.00       0.00
              25%      2184.17     667.01
              50%      3132.70    1256.55
              75%      4119.75    2146.96
              max      4998.12    9613.98

#### CPA

In [58]:
# Step 1: Sjoin of villages and CPAs

cpa_apprch3 = gpd.sjoin(vlgs_appch3, cpa[['CPA_id', 'CPAName_Eg', 'geometry']], predicate='within')

In [59]:
# Step 2: Create a dummy to villages that are within CPAs
 
vlgs_appch3['CPA_dummy'] = np.where(vlgs_appch3['PHUMCODE'].isin(cpa_apprch3['PHUMCODE']), 1, 0)

In [60]:
# Step 3: Calculate distances

vlgs_appch3 = calculate_distances(vlgs_appch3, cpa, 'dist_cpa_m')

In [61]:
vlgs_appch3['treatm_id'] = vlgs_appch3['treatm_id'].fillna(vlgs_appch3['PHUMCODE'])

#### Export files apprch 3

In [ ]:
#control_villages_appch3.to_file(outputs_path + '/control_villages_appch3/control_villages_appch3.shp')

vlgs_appch3.to_file(os.path.join(outputs_path, '/control_villages_appchB/vlgs_appchB.shp'))

/var/folders/rg/s4kgtnsx69j8kmy5pyg5ngrm0000gn/T/ipykernel_99535/29986906.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vlgs_appch3.to_file(outputs_path + '/control_villages_appchB/vlgs_appchB.shp')


### Approach 4

All villages that are within 5km of FA but not close to CFs (<10 km)

In [62]:
%%time
# Step 1: Simplify the geometries and create a 5km buffer around the forests 
fas_buffer_5km = fas.copy()
# 1.1 Simplify geometry to reduce the calculations
fas_buffer_5km['geometry'] = fas_buffer_5km['geometry'].simplify(tolerance=50, preserve_topology=True)
# 1.2 Create buffer for the FAS
fas_buffer_5km['geometry'] = fas_buffer_5km.geometry.buffer(5000)

CPU times: user 1min 17s, sys: 50.3 s, total: 2min 7s
Wall time: 4min 28s


In [63]:
fas_buffer_5km['FID'] = fas_buffer_5km['FID'].astype(str)
fas['FID'] = fas['FID'].astype(str)

In [64]:
# Step 2: Select villages within 5 km of forest areas
villages_within_fas = gpd.sjoin(vlgs2, fas_buffer_5km, predicate='within')

In [65]:
# Step 3: Select villages outside the 10 km buffer of community forests
villages_near_community = gpd.sjoin(vlgs2, cfs_buffer_10km, predicate='within', how='right')
villages_outside_community_buffer = vlgs2[~vlgs2['PHUMCODE'].isin(list(villages_near_community['PHUMCODE']))]

In [66]:
# Step 4: Combine the conditions
control_villages_appch4 = villages_within_fas[villages_within_fas['PHUMCODE'].isin(villages_outside_community_buffer['PHUMCODE'])].reset_index(drop=True)

In [67]:
# Step 5: Calculate distance from the village to the each Fas whose buffer contains the village

    # Merge the two geometries (villages and community forests)
control_villages_appch4 = control_villages_appch4.merge(fas, on="FID", suffixes=("_village", "_fas"))

    # Convert to GeoDataFrame 
control_villages_appch4 = gpd.GeoDataFrame(control_villages_appch4, geometry="geometry_village")

    # Calculate distance between both geometries 
control_villages_appch4["distance_to_forest"] = control_villages_appch4["geometry_village"].distance(gpd.GeoSeries(control_villages_appch4["geometry_fas"], crs=control_villages_appch4.crs))

In [68]:
# Step 6: Drop duplicated villages with the shortest distance to a CF

control_villages_appch4 = control_villages_appch4.sort_values(by=["FID", "distance_to_forest"]).drop_duplicates(subset='PHUMCODE', keep='first').reset_index(drop=True)

In [69]:
# Select only necessary variables 

control_villages_appch4 = control_villages_appch4[['PHUMCODE', 'VILLAGE', 'FID', 'distance_to_forest', 'geometry_village']].copy()

# Rename column to standardize
control_villages_appch4.rename({'geometry_village':'geometry', 'distance_to_forest':'dist_fas'}, axis=1, inplace=True)

In [70]:
    # Convert to GeoDataFrame 
control_villages_appch4 = gpd.GeoDataFrame(control_villages_appch4, geometry="geometry", crs=vlgs.crs)

#### Calculate distances from treatment and control villages

In [71]:
#Create a gdf with treatment and control villages only
control_villages_appch4['Treatment'] = 0

vlgs_appch4 = pd.concat([treatment_villages_gdf, control_villages_appch4])

vlgs_appch4 = vlgs_appch4.reset_index(drop=True)

vlgs_appch4 = vlgs_appch4[['PHUMCODE', 'VILLAGE','Treatment', 'FID', 'dist_fas', 'geometry']]

In [72]:
%%time
# Calculate the distance between the village and the community forest
vlgs_appch4 = calculate_distances(vlgs_appch4, cfs, 'dist_cf_m')

# Calculate the distance between the village and the FAS
vlgs_appch4 = calculate_distances(vlgs_appch4, fas, 'dist_fa_m')

# Calculate the distance between the village and any type of forest
vlgs_appch4 = calculate_distances(vlgs_appch4, forests, 'dist_forest_m')

CPU times: user 19.2 s, sys: 184 ms, total: 19.4 s
Wall time: 19.8 s


In [73]:
vlgs_appch4.drop(['PHUMCODE', 'FID','dist_fas'], axis=1).groupby('Treatment').describe().round(2).transpose()

Treatment                    0          1
dist_cf_m     count     185.00     598.00
              mean    41093.71    1601.89
              std     26257.70    1364.02
              min     10154.26       0.00
              25%     18376.33     667.67
              50%     34486.94    1268.91
              75%     54050.30    2169.25
              max    102656.87    9613.98
dist_fa_m     count     185.00     598.00
              mean     2903.38   23963.89
              std      1359.92   20139.60
              min         0.00     231.33
              25%      2065.55   10051.96
              50%      3046.42   18191.17
              75%      4103.70   32477.26
              max      4970.76  165894.83
dist_forest_m count     185.00     598.00
              mean     2903.38    1565.88
              std      1359.92    1312.31
              min         0.00       0.00
              25%      2065.55     667.01
              50%      3046.42    1256.55
              75%      4103.70    2146.96
              max      4970.76    9613.98

#### CPA

In [74]:
# Step 1: Sjoin of villages and CPAs

cpa_apprch4 = gpd.sjoin(vlgs_appch4, cpa[['CPA_id', 'CPAName_Eg', 'geometry']], predicate='within')

In [75]:
# Step 2: Create a dummy to villages that are within CPAs

vlgs_appch4['CPA_dummy'] = np.where(vlgs_appch4['PHUMCODE'].isin(cpa_apprch4['PHUMCODE']), 1, 0)

In [76]:
# Step 3: Calculate distances

vlgs_appch4 = calculate_distances(vlgs_appch4, cpa, 'dist_cpa_m')

In [77]:
vlgs_appch4.rename({'FID':'FID_fas'}, axis=1, inplace=True)

#### Export files apprch 4

In [350]:
#control_villages_appch4.to_file(outputs_path + '/control_villages_appch4/control_villages_appch4.shp')

vlgs_appch4.to_file(outputs_path + '/control_villages_appchC/vlgs_appchC.shp')

#fas_buffer_5km.to_file(outputs_path + '/fas_buffer_5km/fas_buffer_5km.shp')


/var/folders/rg/s4kgtnsx69j8kmy5pyg5ngrm0000gn/T/ipykernel_99535/958793083.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vlgs_appch4.to_file(outputs_path + '/control_villages_appchC/vlgs_appchC.shp')


# Fuzzy Matching

In [2]:
# import needed packages
from rapidfuzz.process import extractOne
from thefuzz import fuzz, process

In [ ]:
root_path = os.path.join(root_path, 'data')

In [12]:
# define path
os.chdir(root_path)

## Import required datasets

In [4]:
#Updated community forests
cfs_upd = gpd.read_file('Camboda CF updated data - Feb 5, 2025/CF_KH_Updated_2.5.2025_EPSG32648.shp')

#Villages
vlgs = gpd.read_file('KHM-Villages/Villages.shp').to_crs(cfs_upd.crs)

#Adm3
khm_adm3 = gpd.read_file('adm_borders/khm-adm3.shp').to_crs(cfs_upd.crs)

#Treatment villages under the nearest method
nearest_vlgs = gpd.read_file('treatment_contor_proximity/treatment_vlgs_v2.shp').to_crs(cfs_upd.crs)

In [5]:
# Keep relevant columns
cfs_upd = cfs_upd[['ObjectID','CF_Name_En', '#Villages','Village', 'Commune', 'geometry']].copy()
cfs_upd['Commune'] = cfs_upd['Commune'].str.lower()

# Sort values by ObjectID
cfs_upd.sort_values('ObjectID', inplace=True)

In [6]:
cfs_upd_original = cfs_upd.copy()
print(f'There are {str(cfs_upd_original.nunique()['ObjectID'])} community forests in the updated dataset.' )

There are 598 community forests in the updated dataset.


In [7]:
# Keep only those with a village
cfs_upd = cfs_upd[cfs_upd['Village']!='N/A']

In [8]:
print(f'However, only {str(cfs_upd.nunique()['ObjectID'])} had a village assigned.' )

However, only 376 had a village assigned.


## Split columns with multiple villages

In [9]:
vlg_split = cfs_upd['Village'].str.split(',', expand=True) 
vlg_split.columns = [f'Village{i+1}' for i in range(vlg_split.shape[1])]

In [10]:
# Concatenate with original DataFrame, dropping the old 'Values' column
df_melted = pd.concat([cfs_upd.drop(columns=['Village']), vlg_split], axis=1)

In [11]:
# Melt the DataFrame
df_melted = df_melted.melt(id_vars=['ObjectID'], value_vars=['Village1', 'Village2', 'Village3', 'Village4', 'Village5', 'Village6',
       'Village7'], var_name='vil_type', value_name='vil_melt')

# Drop null values
df_melted = df_melted[df_melted['vil_melt'].notnull()].reset_index(drop=True).copy()

# lowercase village names
df_melted['vil_melt'] = df_melted['vil_melt'].str.lower()

In [12]:
cfs_upd = cfs_upd.merge(df_melted, on='ObjectID', how='left')

In [13]:
print(f'Since a single CF can have more than one village assigned, there were {cfs_upd.shape[0]} villages within the CFs area')

Since a single CF can have more than one village assigned, there were 428 villages within the CFs area


## Sjoin between villages and Admin3

The georreferenced village dataset did not have a column containing the Commune. For that reason, a sjoin with a dataset of communes downloaded from https://data.humdata.org was performed. 

In [14]:
# Merge villages dataset with the adm3 (communes) to obtain a commune name 

vlgs = gpd.sjoin(vlgs, khm_adm3[['admin3Name', 'admin3Pcod', 'geometry']], how='left', predicate='within')
vlgs['admin3Name'] = vlgs['admin3Name'].str.lower()
vlgs['VILLAGE'] = vlgs['VILLAGE'].str.lower()
vlgs = vlgs[vlgs['admin3Name'].notnull()]
vlgs.reset_index(drop=True, inplace=True)

In [15]:
# Create a new column with wich the fuzzy match will be performed later on

vlgs['vilcom_vlg'] = vlgs['VILLAGE'] + '-' + vlgs['admin3Name']

## Applying fuzzy matching to village + commune name between the Villages dataset and the CFs dataset

In [16]:
# Create a new column in the CFs dataset with which the fuzzy match will be performed

cfs_upd['Commune'] = cfs_upd['Commune'].str.lower()
cfs_upd['vilcom_cfs'] = cfs_upd['vil_melt'] + '-' + cfs_upd['Commune']

In [17]:
# Apply fuzzy matching between the CFs and the villages datasets, using the variables created before (vill+comm)
results_vlgs_cfs = cfs_upd['vilcom_cfs'].apply(lambda x: extractOne(query = x, choices =vlgs['vilcom_vlg'], score_cutoff=90))

# Create a new column to the CFs table with the result
cfs_upd['match_vlgs_cfs'] = results_vlgs_cfs

# Since there are multiple villages that dont match to any contained in the CFs table, drop the ones that don't match
cfs_upd = cfs_upd[cfs_upd['match_vlgs_cfs'].notna()].copy().reset_index(drop=True)

# Splitting the resulting tuple into three separate columns
cfs_upd[['vilcom_vlg', 'score', 'idx_vlg']] = cfs_upd['match_vlgs_cfs'].apply(pd.Series)

# Drop not relevant variables
cfs_upd.drop('match_vlgs_cfs', axis=1, inplace=True)

# Bring village code from the vlgs dataset, using the index resulting from the fuzzy matching 
cfs_upd = cfs_upd.merge(vlgs[['PHUMCODE']], left_on="idx_vlg", right_index=True, how="left")

In [18]:
cfs_upd.nunique()

ObjectID      268
CF_Name_En    268
#Villages       7
Village       254
Commune       140
geometry      268
vil_type        7
vil_melt      282
vilcom_cfs    295
vilcom_vlg    293
score          45
idx_vlg       293
PHUMCODE      293
dtype: int64

In [17]:
cfs_upd.nunique()

ObjectID      268
CF_Name_En    268
#Villages       7
Village       254
Commune       140
geometry      268
vil_type        7
vil_melt      282
vilcom_cfs    295
vilcom_vlg    293
score          45
idx_vlg       293
PHUMCODE      293
dtype: int64

In [19]:
print(f'There are {cfs_upd.nunique()['PHUMCODE']} villages that matched in both the village dataset and the CFs updated dataset.')

There are 293 villages that matched in both the village dataset and the CFs updated dataset.


In [20]:
print(f'{cfs_upd.nunique()['ObjectID']} Community Forests had a village that matched with the georreferenced village dataset (a CF can have more than 1 village assigned')


268 Community Forests had a village that matched with the georreferenced village dataset (a CF can have more than 1 village assigned


In [24]:
#cfs_upd.to_file('Camboda CF updated data - Feb 5, 2025/cfs_upd_matched.shp')

## Comparing with the "nearest" villages

In [21]:
# Convert data type
cfs_upd['PHUMCODE'] = cfs_upd['PHUMCODE'].astype(str)

In [22]:
# Perform filter based on the village code 
cfs_upd[cfs_upd['PHUMCODE'].isin(nearest_vlgs['PHUMCODE'])].nunique()

ObjectID      127
CF_Name_En    127
#Villages       6
Village       123
Commune        88
geometry      127
vil_type        6
vil_melt      126
vilcom_cfs    129
vilcom_vlg    127
score          34
idx_vlg       127
PHUMCODE      127
dtype: int64

In [23]:
print('There are 127 villages coinciding in both methods')

There are 127 villages coinciding in both methods


In [36]:
# Convert data type
vlgs['PHUMCODE'] = vlgs['PHUMCODE'].astype(str)

In [48]:
# Export matching villages
nearest_vlgs[nearest_vlgs['PHUMCODE'].isin(cfs_upd['PHUMCODE'])].to_file('Camboda CF updated data - Feb 5, 2025/matching_villages.shp')

In [50]:
# Export updated vill ages
vlgs[vlgs['PHUMCODE'].isin(cfs_upd['PHUMCODE'])].to_file('Camboda CF updated data - Feb 5, 2025/updated_villages.shp')

/var/folders/rg/s4kgtnsx69j8kmy5pyg5ngrm0000gn/T/ipykernel_57782/2593421237.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vlgs[vlgs['PHUMCODE'].isin(cfs_upd['PHUMCODE'])].to_file('Camboda CF updated data - Feb 5, 2025/updated_villages.shp')
